In [1]:
import numpy as np
import h5py
import yt

import numpy as np# Build Grid_Data

In [2]:
with open('/Users/yilinxia/Desktop/DXL/yt/IsolatedGalaxy/galaxy0030/galaxy0030.hierarchy') as f:
    lines = f.readlines()

In [3]:
grids_loc=[]
for line in lines:
    if "Grid =" in line:
        grids_loc.append(lines.index(line))

In [4]:
len(lines)

3981

In [5]:
grids_loc[-1]

3947

In [6]:
grids_info=dict()
grid_id=1
for loc_ in range(len(grids_loc)-1):
    start_indx=grids_loc[loc_]
    end_indx=grids_loc[loc_+1]
    grids_info[grid_id]=lines[start_indx:end_indx]
    grid_id+=1
    
grids_info[grid_id]=lines[end_indx:]

In [7]:
grids_info[1]

['Grid = 1\n',
 'Task              = 0\n',
 'GridRank          = 3\n',
 'GridDimension     = 38 38 38 \n',
 'GridStartIndex    = 3 3 3 \n',
 'GridEndIndex      = 34 34 34 \n',
 'GridLeftEdge      = 0 0 0 \n',
 'GridRightEdge     = 1 1 1 \n',
 'Time              = 0.0060000200028298\n',
 'SubgridsAreStatic = 0\n',
 'NumberOfBaryonFields = 24\n',
 'FieldType = 0 4 5 6 1 49 50 51 52 53 7 8 9 10 11 12 23 24 25 27 19 61 62 83 \n',
 'BaryonFileName = ./DD0030/galaxy0030.cpu0000\n',
 'CourantSafetyNumber    = 0.300000\n',
 'PPMFlatteningParameter = 0\n',
 'PPMDiffusionParameter  = 0\n',
 'PPMSteepeningParameter = 0\n',
 'NumberOfParticles   = 0\n',
 'GravityBoundaryType = 1\n',
 'Pointer: Grid[1]->NextGridThisLevel = 0\n',
 'Pointer: Grid[1]->NextGridNextLevel = 2\n',
 '\n']

In [8]:
def extract(line):
    temp=line.split("=")[1].split("\n")[0]
    result=[]
    for item in temp.split(" "):
        if item != "":
            result.append(round(float(item),2))
    return(result)

In [9]:
grid_data=list()

#grids_info is a list
for key, value in grids_info.items():
    i=0
    for line in value:
        if i==0:
            grid_id=int(line.split("=")[1].strip('\n'))
        i+=1
        if "LeftEdge" in line:
            LeftEdge=extract(line)
        if "RightEdge" in line:
            RightEdge=extract(line)
        if "StartIndex" in line:
            StartIndex=extract(line)
        if "EndIndex" in line:
            EndIndex=extract(line)
        if "BaryonFileName" in line:
            FileAdd= line.split("=")[1].split("/")[2].strip('\n')
    Dimensions=np.array(EndIndex)-np.array(StartIndex)+1
    grid_data.append(dict(grid_id=grid_id,left_edge=LeftEdge,right_edge=RightEdge,dimensions=Dimensions,file=FileAdd ))

In [10]:
g=grid_data[0]
g

{'grid_id': 1,
 'left_edge': [0.0, 0.0, 0.0],
 'right_edge': [1.0, 1.0, 1.0],
 'dimensions': array([32., 32., 32.]),
 'file': 'galaxy0030.cpu0000'}

In [11]:
def load_field(g,field):
    cpufile=h5py.File("/Users/yilinxia/Desktop/DXL/yt/IsolatedGalaxy/galaxy0030/{}".format(g['file']))
    output=np.array(cpufile["/Grid%08i" % g['grid_id']][field])
    return(output)

In [12]:
ds=yt.load_sample("IsolatedGalaxy")
level_ls=[]
for i in range(len(ds.index.grids)):
    level_ls.append(ds.index.grids[i].Level)
len(level_ls)

yt : [INFO     ] 2022-03-30 12:04:07,984 Sample dataset found in '/Users/yilinxia/Desktop/DXL/yt/IsolatedGalaxy/galaxy0030/galaxy0030'
yt : [INFO     ] 2022-03-30 12:04:08,072 Parameters: current_time              = 0.0060000200028298
yt : [INFO     ] 2022-03-30 12:04:08,074 Parameters: domain_dimensions         = [32 32 32]
yt : [INFO     ] 2022-03-30 12:04:08,076 Parameters: domain_left_edge          = [0. 0. 0.]
yt : [INFO     ] 2022-03-30 12:04:08,079 Parameters: domain_right_edge         = [1. 1. 1.]
yt : [INFO     ] 2022-03-30 12:04:08,080 Parameters: cosmological_simulation   = 0
Parsing Hierarchy : 100%|██████████████████████████| 173/173 [00:00<00:00, 6728.31it/s]
yt : [INFO     ] 2022-03-30 12:04:08,139 Gathering a field list (this may take a moment.)


173

In [13]:
for g in grid_data: 
    g["density"] = load_field(g,"Density")
    g["level"]= level_ls[g['grid_id']-1]
    if 'grid_id' in g: del g['grid_id']
    if 'file' in g: del g['file']
    g["particle_position_x"] = (np.random.uniform(low=0.25, high=0.75, size=1000), "code_length") # No particles, so set empty arrays
    g["particle_position_y"] = (np.random.uniform(low=0.25, high=0.75, size=1000), "code_length")
    g["particle_position_z"] = (np.random.uniform(low=0.25, high=0.75, size=1000), "code_length")

In [14]:
g.keys()

dict_keys(['left_edge', 'right_edge', 'dimensions', 'density', 'level', 'particle_position_x', 'particle_position_y', 'particle_position_z'])

In [15]:
g["dimensions"]

array([ 8., 12., 12.])

In [16]:
ds = yt.load_amr_grids(grid_data,[32, 32, 32])

YTInconsistentGridFieldShapeGridDims: Not all grid-based fields match the grid dimensions! Grid dims are (18, 16, 16), and the following fields have shapes that do not match them:
    Field ('stream', 'density') has shape (16, 16, 18).
